In [6]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from copy import deepcopy
import requests
from PIL import Image
import os

In [17]:
model = models.MobileNetV2(num_classes=10)

In [18]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [19]:
print_size_of_model(model)

Size (MB): 9.169412


In [21]:
param_0 = next(iter(model.parameters()))
print("Model precision: ", param_0.dtype)


Model precision:  torch.float32


In [22]:
import platform
chip = platform.processor()

if chip == 'arm':
    backend = 'qnnpack'
elif chip in ['x86_64', 'i386']:
    backend = 'fbgemm'
else:
    raise SystemError("Backend is not supported")

print(f"Using {backend} backend engine for {chip} CPU")

torch.backends.quantized.engine = backend

Using fbgemm backend engine for x86_64 CPU


In [28]:
from torch.quantization.quantize_fx import prepare_fx, convert_fx

example_inputs = (torch.randn(1, 3, 32, 32),)
qconfig = {
    "": torch.quantization.get_default_qconfig(backend),
    "module_name": [
#        ("features.13", None),    
#        ("features.14", None),
#        ("features.15", None),
#        ("features.16", None),
#        ("features.17", None),
    ]
}
model_prepared = prepare_fx(model.eval(), qconfig,example_inputs)
dynamic_resnet = convert_fx(model_prepared)

/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/fx/prepare.py:1751: UserWarning: Passing a QConfig dictionary to prepare is deprecated and will not be supported in a future version. Please pass in a QConfigMapping instead.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/observer.py:1272: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [29]:
print_size_of_model(dynamic_resnet)

Size (MB): 2.634946
